In [244]:
import math
import numpy as np
np.set_printoptions(suppress=True) #Evita notación científica

import import_ipynb
import G360_Parameter as param

np.set_printoptions(suppress=True) #Evita notación científica
from collections.abc import Iterable

In [ ]:
# DEFINICIÓN DE FReaction PARA USARLO EN f(x):
class FReaction(np.ndarray):
    def __new__(cls, f_v, f_x=None, f_y=None):
        if (f_x != None) & (f_y != None):
            obj = np.array([f_v, f_x, f_y], dtype=float).view(cls)
        else:
            obj = np.array(f_v, dtype=float).view(cls)
        return obj
        
    def __str__(self):
        return repr(self)

    @property
    def F_vert(self):
        return self[0]

    @property
    def F_x_long(self):
        return self[1]

    @property
    def F_y_transv(self):
        return self[2]

In [4]:
# FUNCIÓN DE INCREMENTO PARA VARIABLES DE CIMENTACIÓN
def incremento(vrbl, incrmnt):
    decimal = vrbl%1
    entero = vrbl//1

    #Generic (0.05, 0.1, 0.5)
    if ( (1/incrmnt)%2 == 0 ):
        vrbl += incrmnt

    # Symmetric by 0.5 (0.2)
    elif ( (1/incrmnt)%1 == 0 ):
        if (0 <= decimal < 0.5) & (decimal + incrmnt > 0.5):
            vrbl = entero + 0.5
        elif (decimal == 0.5):
            vrbl += incrmnt/2
        else:
            vrbl += incrmnt
            
    # error output (0.15, 0.3)         
    else:
        print('no itera')
    
    return vrbl   

In [5]:
# CALCULO DE VOLUMEN PARA PIRAMIDE TRUNCADA
def troncopiramidal(area_inf, area_sup, altura):
    volumen = altura / 3 * (area_inf + area_sup + math.sqrt(area_inf * area_sup))
    return volumen

In [6]:
# REDONDEO EN CANTIDAD DE DECIMALES
def round_up(number, decimals):
    factor = 10 ** decimals
    return math.ceil(number * factor) / factor

def round_down(number, decimals):
    r_r = round_up(number, decimals) - 10**-decimals
    r_r_plus_add = r_r + 10**-decimals
    return r_r_plus_add if (r_r_plus_add == number) else r_r

In [7]:
# REDONDEO EN MULTIPLOS
def mround_up(x, multiple):
    return math.ceil(x / multiple) * multiple

def mround_down(x, multiple):
    r_r = mround_up(x, multiple) - multiple
    return r_r + multiple if math.isclose(x%multiple, multiple) else r_r

In [8]:
# FUNCIÓN DE PROPORCIÓN PARA TRIANGULOS SEMEJANTES
def proporcion(X,Y, y):
    # X/Y == x/y
    return y * X/Y

In [9]:
# ESTA FUNCIÓN RETORNA EL VALOR ABSOLUTO DE LAS FUERZAS CORTANTES DE LOS VECTORES DE LA FORMA [Fv, Fx, Fy] 
def abs_vectores(matrix):
    Copia = matrix.copy()
    len_i = len(Copia)
    len_j = len(Copia[0])
    
    for i in range(len_i):
        for j in range(1, len_j):
            Copia[i][j] = abs(Copia[i][j])
    return Copia

In [8]:
# DEVUELVE UNA MATRIX CON "1" SI LA CARGA VERTICAL ES A COMPRESIÓN O TRACCIÓN
def casos_traccion(np_matrix):
    Lista = [ np.array([0,0,0]) if (i[0] <= 0) else np.array([1,0,0]) for i in np.array(np_matrix) ]
    return np.array(Lista)

def casos_compresion(np_matrix):
    Lista = [ np.array([0,0,0]) if (i[0] > 0) else np.array([1,0,0]) for i in np.array(np_matrix) ]
    return np.array(Lista)

In [3]:
# FUNCIÓN MAXIMOS Y MÍNIMOS QUE IGNORA CADENAS DE TEXTO
def min_(obj):
    _a = obj # OBJETO
    _b = [] # LISTA A EVALUAR
        # QUE SEA UN ITERABLE Y DIFERENTE A UN TEXTO
    while (isinstance(_a[0], Iterable) and not isinstance(_a[0], str)):
        for i in _a:
            for j in i:
                _b.append(j)
        _a = _b
        _b = []
    _b_num = [i for i in _a if isinstance(i, (float, int))]
    return min(_b_num)

def max_(obj):
    _a = obj # OBJETO
    _b = [] # LISTA A EVALUAR
        # QUE SEA UN ITERABLE Y DIFERENTE A UN TEXTO    
    while (isinstance(_a[0], Iterable) and not isinstance(_a[0], str)):
        for i in _a:
            for j in i:
                _b.append(j)
        _a = _b
        _b = []
    _b_num = [i for i in _a if isinstance(i, (float, int))]
    return max(_b_num)

In [12]:
# CALCULO DE EMPUJO PASIVO
def empuje_pasivo(q1, q2, Area, kp):
    return 1/2 * (q1 + q2) * Area * kp

In [13]:
# FUNCIÓN PARA VALORES DEFAULT EN LAS VARIABLES OPCIONALES
def dict_default(funcion):
    try:
        _r = funcion()
    except Exception:
        _r = None
    return _r

In [14]:
# LONGITUD DE TEXTO EN DICCIONARIOS; PARA ALINEAMIENTO
def len_values(diccionario):
    _len_max = 0
    for k,v in diccionario.items():
        _len_max = len(v) if (len(v)>_len_max) else _len_max
    return _len_max

In [15]:
# CONVERSIÓN DE MATRIZ EQUIVALENTE PARA CHILE CONSIDERANDO LA EXCENTRICIDAD
def Fx_y_equivalencias(num_pata, vector, excentricidad, altura):
    np.set_printoptions(suppress=True)
    _m = []
    #1
    if num_pata == 1:
        for i in vector:
            _v = np.array([+ i.F_vert,
                           + i.F_vert * excentricidad/altura + i.F_x_long, 
                           - i.F_vert * excentricidad/altura - i.F_y_transv ])
            _m.append(_v)
    #2
    elif num_pata == 2:
        for i in vector:
            _v = np.array([+ i.F_vert,
                           + i.F_vert * excentricidad/altura + i.F_x_long, 
                           + i.F_vert * excentricidad/altura - i.F_y_transv ])
            _m.append(_v)
    #3
    elif num_pata == 3:
        for i in vector:
            _v = np.array([+ i.F_vert,
                           - i.F_vert * excentricidad/altura + i.F_x_long, 
                           + i.F_vert * excentricidad/altura - i.F_y_transv ])
            _m.append(_v)        
    #4
    elif num_pata == 4:
        for i in vector:
            _v = np.array([+ i.F_vert,
                           - i.F_vert * excentricidad/altura + i.F_x_long, 
                           - i.F_vert * excentricidad/altura - i.F_y_transv ])
            _m.append(_v)        
    #0
    else:
        for i in vector:
            _v = np.array([+ i.F_vert,
                           + abs(i.F_vert) * excentricidad/altura + abs(i.F_x_long), 
                           + abs(i.F_vert) * excentricidad/altura + abs(i.F_y_transv) ])
            _m.append(_v)    
    return np.array(_m)

def matriz_equivalente(altura, excentricidad, num_pata_c, num_pata_t, vector):
    f_c = Fx_y_equivalencias(altura = altura, 
                       excentricidad = excentricidad, 
                       num_pata = num_pata_c, 
                       vector = vector)
    f_t = Fx_y_equivalencias(altura = altura, 
                       excentricidad = excentricidad, 
                       num_pata = num_pata_t, 
                       vector = vector)
    _mtrx = []
    for i in range(4):
        if vector[i].F_vert <= 0:
            _mtrx.append(f_c[i])
        else:
            _mtrx.append(f_t[i])
    _mtrx = FReaction(_mtrx)
    return _mtrx

In [16]:
# FORMULAS Y TABLAS PARA MÉTODO MeyerhofAdams
def m_MeyerhofAdams(_phi):
        # f	    m
    m = {'20': 0.05,
         '25': 0.10,
         '30': 0.15,
         '35': 0.25,
         '40': 0.35}
    _v = 0
    for k,v in m.items():
        _v = v if _phi >= int(k) else _v
    return _v

def ku_MeyerhofAdams(_phi):
    return 0 if _phi < 30 else 0.126 * math.log(_phi) +0.4857

def Qu_MeyerhofAdams(Coh, Ancho, Prof, eme, gamma_s, phi, Wcr, Wsaport):
    return math.pi * Coh * Ancho * Prof + ( (1 + eme * Prof/Ancho) * math.pi/2 * gamma_s * Prof**2 * Ancho *eme * math.tan(phi) ) + Wcr + Wsaport

In [1]:
# CONVERSIÓN A ANGULO TRIDIMENSIONAL
def ang_bid_to_trid(ang_bid):
    ang_trid = math.atan( math.sqrt(2) * math.tan(ang_bid) )
    return ang_trid

In [4]:
# CENTROIDE PARA UN TRAPECIO, APORTE kp LOSA
def cg_trapecio(b, B, h):
    cg = (h / 3) * ((b + 2 * B) / (b + B))
    return cg

In [204]:
# ECUACIONES DE MOMENTOS PARA VIGAS Equiv
def Mom_viga_voladizo(ω , luz):
    return ω * luz**2 /2

def Mom_viga_simple_apoyada(ω , luz):    
    return ω * luz**2 /8

def Ref_flexotrac(Ps, Mc2, D, recub, fy, fc):
    Mm = (Ps * 10)/(0.9 * fy) + (0.85*fc)/(fy) * ( 1 -np.sqrt( 1- 2 * (fy * Mc2)/(.85 * fc * 0.9 * D * (D*.8-recub)**2  * 1000 * fy) ))  * D * (.8 * D-recub) * 2*10000
    return Mm

def Mom_viga_esbelta(Geometria, fprima_c, fy, ancho, rr, luz, Vector):
#PARAMETROS
    ME = 4700 * math.sqrt(fprima_c)*1000 
    MI = ancho**4/12 if Geometria==0 else math.pi * (ancho/2)**4 /4
    rgiro = (0.3 if Geometria==0 else 0.25)*ancho
    flong_eff = 2.1
    B_nds = 0.6
    _phi = 0.75
    EIeff = 0.4 * MI * ME /(1+B_nds)

# MODIFICAR VECTOR DE REACCIONES
    # PESO DEL PEDESTAL
    Vector -= np.array([24 * ancho**2 * luz  ,0,0])
    
    # REACCION EN x, y
    if Geometria==1 :
        Vector_xy = np.sqrt(Vector[:,1]**2 + Vector[:,2]**2)
    else:
        Vector_xy = np.maximum(abs(Vector[:,1]), abs(Vector[:,2]))
    Vector[:,1] = Vector_xy
    Vector = Vector[:, :-1]

    # AGREGAR MOMENTO HASTA EL PEDESTAL
    Vector = np.column_stack( (Vector,  Vector_xy * luz  ))
    
    #
    Esbeltez = flong_eff * luz / rgiro > 22
    if Esbeltez == True:
        # EFECTOS DE ESBELTEZ
        Pc = math.pi**2 *EIeff/(flong_eff * luz)**2
        Ps = abs( Vector * casos_compresion(Vector) )[:,0]
        
        delta = 1/(1- Ps/(_phi * Pc))
        Mc = delta * Vector[:,2]
        Delta = Vector_xy * luz**3 / (3 * EIeff) * 1000
        M2o = Delta * Ps/1000
        Mc2 = M2o + Mc

        Vector[:,2] = Mc2        
    return Vector

In [ ]:
# ESFUERZOS σ DE COMPRESIÓN EN EL SUELO
def sigma_comp(Fv, Ancho, exc):
    L_sigma = []
    for i in range(4):
        sigma = []
    
        sigma.append(Fv[i]/Ancho**2 * (Ancho + 6 * exc[i][0] + 6 * exc[i][1])/Ancho)
        sigma.append(Fv[i]/Ancho**2 * (Ancho + 6 * exc[i][0] - 6 * exc[i][1])/Ancho)
        sigma.append(Fv[i]/Ancho**2 * (Ancho - 6 * exc[i][0] + 6 * exc[i][1])/Ancho)
        sigma.append(Fv[i]/Ancho**2 * (Ancho - 6 * exc[i][0] - 6 * exc[i][1])/Ancho)
        
        sigma = np.array(sigma)
        L_sigma.append(sigma)
        
    return np.array(L_sigma)

In [246]:
param.BARRAS_mm

array([['No.3', '9.5', '70.9', '0.56', '57.0', '0.8'],
       ['No.4', '12.7', '126.7', '0.994', '76.2', '0.8'],
       ['No.5', '15.9', '198.6', '1.552', '95.4', '0.8'],
       ['No.6', '19.1', '286.5', '2.235', '114.6', '0.8'],
       ['No.7', '22.2', '387.1', '3.042', '133.2', '1.0'],
       ['No.8', '25.4', '506.7', '3.973', '152.4', '1.0'],
       ['No.9', '28.6', '641.4', '5.067', '171.45', '1.0'],
       ['No.10', '31.8', '791.8', '6.255', '190.5', '1.0'],
       ['No.11', '34.9', '958.0', '7.568', '209.55', '1.0']], dtype='<U32')

In [17]:
# MÉTODO XY - TRANSELEC
def Met_XY_excentricidad_comp(Permanentes, Eventuales, Ancho, Altura, Wc, Wconcr):
    # CASO 1
    _momentos = Permanentes * np.array([-1, Altura, Altura])+ np.array([3/4*Wc + Wconcr, -1/8 * Ancho * Wc, -1/8 * Ancho * Wc])
    _exc = np.array([i/i.F_vert   for i in _momentos])
    _exc = np.array([[0,0,0,0], _exc[:,2], _exc[:,1]]).T
    _exc1 = [ sum(i) <  Ancho/6  for i in _exc]
    _exc1 = _exc
    #
    _momentos = Eventuales * np.array([-1, Altura, Altura])+ np.array([3/4*Wc + Wconcr, -1/8 * Ancho * Wc, -1/8 * Ancho * Wc])
    _exc = np.array([i/i.F_vert   for i in _momentos])
    _exc = np.array([[0,0,0,0], _exc[:,2], _exc[:,1]]).T
    _exc2 = [ sum(i) <  Ancho/6  for i in _exc]
    _exc2 = _exc
    
    # CASO 2
    #AGREGAR CASO 2

    return [_exc1, _exc2, _exc3, _exc4]

#Jc_Flag ACTUALIZAR FORMULA DE TRACCIÓN Y CREAR CASO DOS PARA COMPRESIÓN SEGÚN TRANSELEC
def Met_XY_excentricidad_trac(Permanentes, Eventuales, Ancho, Altura, Wc, Wconcr):
    # CASO 1
    _momentos = Permanentes * np.array([-1, Altura, Altura])+ np.array([3/4*Wc + Wconcr, -1/8 * Ancho * Wc, -1/8 * Ancho * Wc])
    _exc = np.array([i/i.F_vert   for i in _momentos])
    _exc = np.array([[0,0,0,0], _exc[:,2], _exc[:,1]]).T
    _exc1 = [ sum(i) <  Ancho/6  for i in _exc]